In [632]:
data = rand(1000, 5) .- 0.5;
x_train = data[1:750, :];
y_train = vec(sum(map.(x->x^3, x_train), dims=2));

x_test = data[751:end, :];
y_test = vec(sum(map.(x->x^3, x_test), dims=2));

In [685]:
config = EvoTreeRegressor(nrounds=3, max_depth=5);
evo_model = fit_evotree(config; x_train, y_train, verbosity=0);
evo_model_temp = evo_model
evo_model = evo_model.trees[2]

EvoTrees.Tree{EvoTrees.MSE, 1}
 - feat: [5, 2, 3, 3, 3, 1, 4, 1, 4, 2, 4, 2, 5, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 - cond_bin: UInt8[0x10, 0x0d, 0x33, 0x04, 0x0e, 0x35, 0x34, 0x1b, 0x19, 0x26, 0x27, 0x0b, 0x3e, 0x0f, 0x3a, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00]
 - cond_float: Any[-0.2504214846695607, -0.29226347146649284, 0.31860200364916236, -0.44524407733050814, -0.27904247754326855, 0.33281540539685156, 0.29926455245624184, -0.09305782079637451, -0.11393074036259893, 0.10833475297674695, 0.10030039051040862, -0.3314363590602485, 0.46491940736592585, -0.2665984969661571, 0.41573503481621343, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 - gain: [1.3125857750311107, 0.44825225792196455, 0.9623697654853955, 0.11481583554934849, 0.1969904513561656, 0.601191074711292, 0.3239248591184525, 0.01529426635542308, 0.08628197872874199, 0.04020464065320806, 0.1259515356732703, 0.432828

In [686]:
n_trees = 1
n_feats = 5


# Get number of leaves and ids of the leaves on each tree
leaves = findall(node -> evo_model.split[node] == false && (node == 1 || evo_model.split[floor(Int, node / 2)] == true), 1:length(evo_model.split))
n_leaves = length(leaves)

splits = Matrix{Any}(undef, n_trees, length(evo_model.split)) # storing the feature number and splitpoint index for each split node
splits_ordered = Array{Vector}(undef, n_feats) # splitpoints for each feature

n_splits = zeros(Int64, n_feats)
[splits_ordered[feat] = [] for feat in 1:n_feats]

for node in eachindex(evo_model.split)
    if evo_model.split[node] == true
        splits[node] = [evo_model.feat[node], evo_model.cond_float[node]] # save feature and split value
        push!(splits_ordered[evo_model.feat[node]], evo_model.cond_float[node]) # push split value to splits_ordered
    end
end

[unique!(sort!(splits_ordered[feat])) for feat in 1:n_feats] # sort splits_ordered and remove copies
[n_splits[feat] = length(splits_ordered[feat]) for feat in 1:n_feats] # store number of split points

for node in eachindex(evo_model.split)
    if evo_model.split[node] == true
        
        feature::Int = splits[ node][1]
        value = splits[node][2]

        splits[node][2] = searchsortedfirst(splits_ordered[feature], value)

    end
end

a = zeros(n_leaves, n_feats) 
b = evo_model.pred

split_nodes = evo_model.split; 

In [704]:
model = Model(() -> Gurobi.Optimizer());
set_attribute(model, "OutputFlag", 0) # JuMP or solver-specific attributes can be changed

L_bounds_inp = -0.5*ones(n_feats);
U_bounds_inp = 0.5*ones(n_feats);

@variable(model, z[l = 1:n_leaves], Bin); 
@variable(model, y[i = 1:n_feats, 1:n_splits[i]], Bin);
@variable(model, x[1:n_feats]);

@constraint(model, [i = 1:n_feats, j = 1:(n_splits[i]-1)], y[i,j] <= y[i, j+1]); 
@constraint(model, [i = 1:n_feats], x[i] <= U_bounds_inp[i]);
@constraint(model, [i = 1:n_feats], x[i] >= L_bounds_inp[i]);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-20


In [705]:
v = []

for i = 1:length(splits_ordered)
    push!(v, vcat(L_bounds_inp[i], splits_ordered[i], U_bounds_inp[i]))
end

In [706]:
@constraint(model, [i = 1:n_feats], x[i] >=  v[i][1] + sum((v[i][j] - v[i][j-1])*(1 - y[i, j-1]) for j = 2 : n_splits[i]+1))
@constraint(model, [i = 1:n_feats], x[i] <=  v[i][n_splits[i]+2] + sum((v[i][j] - v[i][j+1])* y[i, j-1] for j = 2 : n_splits[i]+1))
@constraint(model, sum(z[i] for i=1:n_leaves) == 1)
@objective(model, Max, sum((sum(a[i, :] .* x) +b[leaves][i]) * z[i] for i=1:n_leaves));

In [707]:
model

A JuMP Model
Maximization problem with:
Variables: 36
Objective function type: QuadExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 10 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 20 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 31 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: x, y, z

In [708]:
optimize!(model)

In [709]:
objective_value(model)

0.024467481300234795

In [710]:
value.(model[:z])

16-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0

In [711]:
value.(model[:x])

5-element Vector{Float64}:
 0.5
 0.5
 0.5
 0.5
 0.5

In [712]:
sum((a * value.(model[:x]) + b[leaves]) .*  value.(model[:z]))

0.024467481300234795

In [713]:
b[leaves] 

16-element Vector{Float32}:
 -0.030662792
 -0.020315818
 -0.019357136
 -0.009705952
 -0.015093801
 -0.008083879
 -0.0050189067
  0.0020153637
 -0.009251712
  0.00010026542
  0.0068870354
  0.023888988
  0.00036674537
  0.009923325
  0.015579635
  0.024467481

In [714]:
EvoTrees.predict(evo_model_temp, value.(model[:x])')

1-element Vector{Float32}:
 0.061836593